# Activation patching basic


In [ ]:

%load_ext autoreload
%autoreload 2

device = "cuda:0"
starcoderbase_1b = "/home/arjun/models/starcoderbase-1b/"

from nnsight import LanguageModel
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from src.utils import *
import datasets
from src.experiments.wrong_type_patching import *
from src.experiments.type_patching import *
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import plotly.express as px
from src.patching import *
from nnsight import LanguageModel, util
from nnsight.tracing.Proxy import Proxy
import pandas as pd
import datasets
from src.utils import *
import regex as re
import circuitsvis as cv

In [ ]:
with open("causal_mask_epoch_1.pt","rb") as f:
    causal_mask = torch.load(f)
    
print(causal_mask)

In [ ]:
model = LanguageModel(starcoderbase_1b, device_map=device)
model, model.config

## Logits for inf binding

In [ ]:
dataset = datasets.load_dataset("franlucc/ts_bench_starcoder1b_funcfim_incorrect_uniq_v1", split="train")


In [ ]:
count = 0
seen = set()
model.tokenizer.pad_token_id = model.tokenizer.eos_token_id
correct = []
incorrect = []

for i,ex in tqdm(enumerate(dataset), total=len(dataset)):
    if i > 10:
        break
    prompt = ex["prompt"]
    if prompt in seen:
        print("Skipping ", i)
        continue
    seen.add(prompt)
    def decoder(x):
        # decoder consists of final layer norm + unembedding
        return model.lm_head(model.transformer.ln_f(x))

    logits_before = []
    logits = []
    with model.generate(max_new_tokens=2) as generator:
        with generator.invoke(prompt) as invoker:
            tokens = invoker.input["input_ids"]
            for l in model.transformer.h:
                logits_before.append(decoder(l.output[0]).save())
            invoker.next()

            for l in model.transformer.h:
                logits.append(decoder(l.output[0]).save())
                

    logits = util.apply(logits, lambda x: x.value.cpu(), Proxy)
    logits_before = util.apply(logits_before, lambda x: x.value.cpu(), Proxy)

    logits = [torch.cat((a, b), dim=1) for (a,b) in zip(logits_before, logits)]
    logits = [l.cpu().detach().softmax(-1) for l in logits]

    tokens = torch.cat((tokens[0], logits[-1][:,-2,:].argmax(-1))) #-2 here
    
    def to_string(x):
        s =  model.tokenizer.decode([x])
        if s in ["<pad>", "<unk>", "<s>", "</s>", "<fim_prefix>", "<fim_suffix>", "<fim_middle>"]:
            s = "*" + s[1:-1] + "*"
        return s
    
    l = 18

    for e,tok in enumerate(tokens):
        if tok == model.tokenizer.encode("<fim_suffix>")[0]:
            max_logit = logits[l][:,e-1,:].max(-1)
            max_logit_i = max_logit.indices.item()
            max_logit_v = max_logit.values.item()
            max_logit_i = model.tokenizer.decode(max_logit_i)
            if int(max_logit_i) == int(ex["fim_sol"]):
                print("Correct idx", i)
                correct.append((i, max_logit_v))
                count += 1
            else:
                max_logit = logits[l][:,-2,:].max(-1)
                max_logit_i = max_logit.indices.item()
                max_logit_v = max_logit.values.item()
                max_logit_i = model.tokenizer.decode(max_logit_i)
                if int(max_logit_i) == int(ex["fim_sol"]):
                    print("Correct idx", i)
                    correct.append((i, max_logit_v, -1))
                    print("Correct idx mid", i)
                else:
                    incorrect.append((i, max_logit_v, max_logit_i, logits[l][:,e-1,model.tokenizer.encode(ex["fim_sol"])[0]].item()))
                    print("Incorrect idx", i)
                    

In [ ]:
len(incorrect), len(correct), len(correct) / (len(incorrect) + len(correct)), len(seen)

In [ ]:
from IPython.display import Code
from circuitsvis.logits import token_log_probs
k= -2
i = incorrect[k][0]
prompt_fim = dataset[i]["prompt"]
print("correct", dataset[i]["fim_sol"], " incorrect", dataset[i]["generated_text"])

In [ ]:
Code(prompt_fim, language="typescript")

In [ ]:
def decoder(x):
  # decoder consists of final layer norm + unembedding
  return model.lm_head(model.transformer.ln_f(x))

logits_before = []
logits = []
with model.generate(max_new_tokens=2) as generator:
    with generator.invoke(prompt_fim) as invoker:
        tokens = invoker.input["input_ids"]
        for l in model.transformer.h:
            logits_before.append(decoder(l.output[0]).save())
        invoker.next()

        for l in model.transformer.h:
            logits.append(decoder(l.output[0]).save())
            

logits = util.apply(logits, lambda x: x.value.cpu(), Proxy)
logits_before = util.apply(logits_before, lambda x: x.value.cpu(), Proxy)

print(logits[0].shape, logits_before[0].shape)

logits = [torch.cat((a, b), dim=1) for (a,b) in zip(logits_before, logits)]
logits = [l.cpu().detach().softmax(-1) for l in logits]
print(logits[-1].shape)
print([to_string(t) for t in tokens[0]])
tokens = torch.cat((tokens[0], logits[-1][:,-2,:].argmax(-1))) #-2 here

In [ ]:
def to_string(x):
    s =  model.tokenizer.decode([x])
    if s in ["<fim_prefix>", "<fim_suffix>", "<fim_middle>"]:
        s = "*" + s[1:-1] + "*"
    return s
  
l = 16
print([to_string(t) for t in tokens])
print("correct", dataset[i]["fim_sol"], " incorrect", dataset[i]["generated_text"])
for e,tok in enumerate(tokens):
    if tok == model.tokenizer.encode("<fim_suffix>")[0]:
        max_logit = logits[l][:,e-1,:].max(-1)
        max_logit_v = max_logit.values.item()
        max_logit = model.tokenizer.decode(max_logit.indices.item())
        print(max_logit, max_logit_v)
        # if int(max_logit) == int(dataset[i]["fim_sol"]):
        #     print("Correct idx", i)
        #     count += 1
    

# token_log_probs(tokens, torch.log(logits[l]), to_string)

## Investigating what happens when a model gives the wrong prediction

Here we give an approximate algorithm of what the model may be doing:

1. identify first untyped variable and bind it to a value of the same type by transfering info from value to variable
2. repeat 1 until all variables are typed
3. copy the answer onto `<fim_middle>`

In [ ]:
ds = datasets.load_dataset("franlucc/starcoderbase-1b-wrong-type-patching-v0-seed42", split="train")
working_ds = ds.filter(lambda x: x["success"])
broken_ds = ds.filter(lambda x: not x["success"])
ds, working_ds, broken_ds

## looking at one failing example

In [ ]:
from IPython.display import Code
content = broken_ds[0]["prompt"]
solution = broken_ds[0]["solution"]
generated = broken_ds[0]["generated"]
print(solution, "\n\n", generated)
content = content.split("function _uniq_7")[0]

rename = {
    "_uniq_0": "_uniq_q",
    "_uniq_1[^\d]": "_uniq_t",
}
def rename_vars(content, rename):
    for old, new in rename.items():
        content = re.sub(old, new, content)
    return content

content = rename_vars(content, rename)
solution = rename_vars(solution, rename)
generated = rename_vars(generated, rename)
print("SOLUTION:", solution, "\n\n", "GENERATED:", generated)

code = Code(content, language="typescript")
base_prompt = placeholder_to_std_fmt(content, STARCODER_FIM)
code

In [ ]:
prompt = "function" +base_prompt.split("function")[-1]

max_new_tokens = 4

def get_final_states(prompt, max_new_tokens):
    with model.generate(max_new_tokens=max_new_tokens) as generator:
        with generator.invoke(input=prompt) as invoker:
            hidden_states = []
            for i in range(max_new_tokens):
                final_hs = model.lm_head.output
                print(model.transformer.h[5].mlp.output.shape)
                hidden_states.append(final_hs.save())
                invoker.next()
    return hidden_states

hidden_states = get_final_states(prompt, max_new_tokens)

In [ ]:
out = [util.apply(i, lambda x: x.value, Proxy) for i in hidden_states]

topk=10
predictions = []
for j,hs in enumerate(out):
    if j == 0:
        hs = hs[:, -1, :]
        hs = hs.unsqueeze(1)
    top_vals_and_idx = hs.softmax(dim=-1).topk(topk)
    indices = top_vals_and_idx.indices[0][0].tolist()
    values = top_vals_and_idx.values.tolist()[0][0]
    top_idx = [model.tokenizer.decode(i) for i in indices]
    predictions.append(list(zip(top_idx, values)))

greedy_prediction = "".join([p[0][0] for p in predictions])
print(greedy_prediction)
result_table = pd.DataFrame(predictions, columns=[f"top_pred_{i}" for i in range(topk)], index=[f"next_token_{i}" for i in range(max_new_tokens)])
result_table

In [ ]:
## Sanity check generation

# from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained(starcoderbase_1b)
# llm = AutoModelForCausalLM.from_pretrained(starcoderbase_1b).to(device)

In [ ]:
# max_new_tokens = 4
# tokens = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
# out = llm.generate(tokens, max_new_tokens=max_new_tokens, do_sample=False)
# text = [tokenizer.decode(i) for i in out.tolist()]
# text = "".join(text)
# text.split("<fim_middle>")[-1]

In [ ]:
saved_tokens = []
def get_all_hidden_states(prompt, max_new_tokens):
    with model.generate(max_new_tokens=max_new_tokens) as generator:
        with generator.invoke(input=prompt) as invoker:
            saved_tokens.append(invoker.input.tokens())
            print(saved_tokens)
            hidden_states = {layer : [] for layer in range(model.config.n_layer)}
            for i in range(max_new_tokens-1):
                invoker.next()
                
            for l in range(model.config.n_layer):
                hs = model.transformer.h[l].output[0].save()
                hidden_states[l].append(model.lm_head(model.transformer.ln_f(hs)).save())
    return hidden_states

full_hidden_states = get_all_hidden_states(prompt, max_new_tokens)
value_hidden_states = {k : [util.apply(i, lambda x: x.value, Proxy) for i in v] for k,v in full_hidden_states.items()}


In [ ]:
tok_preds = {layer : [] for layer in range(model.config.n_layer)}
# value_hidden_states[14]
for layer, hs in value_hidden_states.items():
    tok_hs = hs[0]
    top_vals_and_idx = tok_hs.softmax(dim=-1).topk(topk)
    indices = top_vals_and_idx.indices.tolist()[0][0]
    values = top_vals_and_idx.values.tolist()[0][0]
    top_idx = [model.tokenizer.decode(i) for i in indices]
    tok_preds[layer].append(list(zip(top_idx, values)))
    
tok_preds_df = pd.DataFrame(tok_preds)
tok_preds_df.columns = [f"layer_{i}" for i in range(model.config.n_layer)]
tok_preds_df

In [ ]:
## look at logits in prompt and how they change through layers
# for each layer, collect the topk predictions for each token in the prompt

prompt_tok_preds = {l : [] for l in range(model.config.n_layer)}
for layer, hs in value_hidden_states.items():
    prompt_hs = hs[0]
    for k in range(prompt_hs.shape[1]):
        tok_hs = prompt_hs[:, k, :]
        top_vals_and_idx = tok_hs.softmax(dim=-1).topk(topk)
        indices = top_vals_and_idx.indices[0].tolist()
        values = top_vals_and_idx.values.tolist()[0]
        top_idx = [model.tokenizer.decode(i) for i in indices]
        prompt_tok_preds[layer].append(list(zip(top_idx, values)))

In [ ]:
prompt_logits = pd.DataFrame(prompt_tok_preds)
prompt_logits.columns = [f"layer_{i}" for i in range(model.config.n_layer)]
prompt_logits

In [ ]:
# curr_layer = prompt_logits["layer_23"].tolist()
# print(len(curr_layer))

# probabilities = []
# text = []
# x = []
# minn = 251
# for k in range(minn, minn+10):
#     token = curr_layer[k]
#     token_logit_text = [i[0] for i in token]
#     token_logit_prob = [i[1] for i in token]
#     probabilities.append(token_logit_prob)
#     x.append(saved_tokens[0][k])
#     text.append(token_logit_text)
    
# import plotly.graph_objects as go

# fig = go.Figure(data=go.Heatmap(
#                     z=probabilities,
#                     x = x,
#                     text=text,
#                     texttemplate="%{text}",
#                     textfont={"size":15}))


# fig.show()

Reasons why prediction is wrong
- 0 is similar to 1 (number issue)
    - probably not, can change 0 to q and 1 to t and it still fails
- another algorithm picks 1

In [ ]:

print(f"====Generated:====\n\n{generated}\n\n====Solution:=====\n\n{solution}")
code

# Attn visualization

In [ ]:
# TODO something wrong here, always gives same resd
attn, toks = attention_vis(model, prompt+"_uniq_")
print(len(attn), torch.stack(attn).sum().item())
# b, h, q, k

In [ ]:
from sklearn.preprocessing import normalize


def normalize_attn(attention_matrix):
    return normalize(attention_matrix, norm="l2", axis=1)

print(attn[0].shape)
normalize_attn(attn[13][3].cpu().detach().numpy()), attn[13][3]

In [ ]:
# cv.attention.attention_pattern(tokens=no_special_tok[window_idx:], attention=attn[14][window_idx:], mask_upper_tri=True)
# query is destination, key is source

# heatmap
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(15,15))

def create_attention_heatmap(attention_matrix, tokens, row_idx, column_idx, step=10):
    normalized_attention = normalize_attn(attention_matrix)
    
    # Get a subset of the attention matrix
    subset_matrix = normalized_attention[row_idx:row_idx+step, column_idx:column_idx+step]
    xtokens = tokens[column_idx:column_idx+step]
    ytokens = tokens[row_idx:row_idx+step]

    
    # Create a heatmap using Seaborn
    sns.set()

    sns.heatmap(subset_matrix, cmap="viridis", xticklabels=xtokens, yticklabels=ytokens)
    # annot=subset_matrix, fmt=".2f", annot_kws={"size": 10}
    
    plt.xlabel("Quert DST Tokens")
    plt.ylabel("Key SRC Tokens")



layers = []
heads = list(range(16))
window_range = list(range(0, len(toks)-10, 10))

column_idx = len(toks)-10
step = 10
full_sum = 0

os.makedirs("images", exist_ok=True)
    
for layer in tqdm(layers):
    os.makedirs(f"images/layer_{layer}", exist_ok=True)

    for row_idx in window_range:
        xtokens = {i:t for i,t in enumerate(toks[column_idx:column_idx+step])}
        ytokens = {i:t for i,t in enumerate(toks[row_idx:row_idx+step])}
        # print("X DST:\n",xtokens, "\nY SRC:\n", ytokens)
        for h in heads:
            # create a subplot for this head    
            plt.subplot(4,4, h+1)
            kq = attn[layer][h].cpu().detach().numpy()
            
            summ = kq[row_idx:row_idx+step, column_idx:column_idx+step].sum().item()
            if summ == 0:
                continue
            full_sum += summ
            create_attention_heatmap(kq, toks, row_idx=row_idx, column_idx=column_idx, step=step)
        
        if full_sum > 0:
            # print(f"Saving {h} head at row {row_idx}")
            full_sum = 0
            # save
            plt.tight_layout()
            with open(f"images/layer_{layer}/row_{row_idx}_column_{column_idx}_step_{step}.txt", "w") as f:
                f.write(f"X DST:\n{xtokens}\nY SRC:\n{ytokens}")
            plt.savefig(f"images/layer_{layer}/row_{row_idx}_column_{column_idx}_step_{step}.png")
        


## Layer erase - logit difference

erasing doesn't quite work -> maybe try noise or random patch?

In [ ]:
def get_final_states_erase(prompt, max_new_tokens, layers = []):
    with model.generate(max_new_tokens=max_new_tokens) as generator:
        with generator.invoke(input=prompt) as invoker:
            hidden_states = []
                    
            for i in range(max_new_tokens):

                for l in range(model.config.n_layer):
                    if l in layers:
                        if l == 0:
                            raise ValueError("Cannot erase layer 0")
                        else:
                            model.transformer.h[l].output = last_saved
                    else:
                        last_saved = model.transformer.h[l].output.save()
                final_hs = model.lm_head.output
                hidden_states.append(final_hs.save())
                invoker.next()
             
            hidden_states.append(final_hs.save())
            
    return hidden_states


hidden_states = get_final_states(prompt, 4)
hidden_states_erase = get_final_states_erase(prompt, 4, layers=list(range(2,24)))

out = [util.apply(i, lambda x: x.value, Proxy) for i in hidden_states]
out_erase = [util.apply(i, lambda x: x.value, Proxy) for i in hidden_states_erase]

logit_diffs_q = []
logit_diffs_t = []

q_idx = model.tokenizer.encode("q")[0]
t_idx = model.tokenizer.encode("t")[0]

topk=10
predictions = []
for j in range(len(out)):
    hs = out[j]
    hs_erase = out_erase[j]
    if j == 0:
        hs = hs[:, -1,:]
        hs = hs.unsqueeze(1)
        hs_erase = hs_erase[:, -1,:]
        hs_erase = hs_erase.unsqueeze(1)
        
    logits_clean = hs.softmax(dim=-1)
    logits_erase = hs_erase.softmax(dim=-1)
    
    logit_clean_q = logits_clean[:,:,q_idx].item()
    logit_erase_q = logits_erase[:,:,q_idx].item()
    logit_clean_t = logits_clean[:,:,t_idx].item()
    logit_erase_t = logits_erase[:,:,t_idx].item()
    
    max_logit_clean = logits_clean.argmax().item()
    max_logit_erase = logits_erase.argmax().item()
    
    logit_diffs_q.append([logit_clean_q - logit_erase_q, logit_clean_q, logit_erase_q])
    logit_diffs_t.append([logit_clean_t - logit_erase_t, logit_clean_t, logit_erase_t])
    
erase_prediction = model.tokenizer.decode(max_logit_erase)
clean_prediction = model.tokenizer.decode(max_logit_clean)
print(f"Clean: {clean_prediction}, Erase: {erase_prediction}")

result_table = pd.DataFrame([logit_diffs_q[-1], logit_diffs_t[-1]], index=["q","t"], columns=["diff", "clean", "erase"])
result_table